In [1]:
# import necessary packages
import pandas as pd
import numpy as np

In [3]:
# loading data
df_daily_hr = pd.read_csv("../Data/patient_1/patient_1_daily_hr.csv")
df_activity = pd.read_csv("../Data/patient_1/patient_1_activity.csv")
df_hr_hrv = pd.read_csv("../Data/patient_1/patient_1_hr_hrv.csv")
df_readiness = pd.read_csv("../Data/patient_1/patient_1_readiness.csv")
df_sleep = pd.read_csv("../Data/patient_1/patient_1_sleep.csv")
df_sleep_periods = pd.read_csv("../Data/patient_1/patient_1_sleep_periods.csv")

In [4]:
df_daily_hr.head(5)

,timestamp,bpm,quality,source,restorative
0,2021-11-28T16:41:28.000Z,65,bad,awake,False
1,2021-11-28T16:41:29.000Z,70,bad,awake,False
2,2021-11-28T16:41:30.000Z,75,bad,awake,False
3,2021-11-28T16:51:32.000Z,77,bad,awake,False
4,2021-11-28T16:51:35.000Z,73,bad,awake,False


Exploratory Data Analysis (EDA)

In [5]:
# Convert timestamp cloumn into pandas datetime type
df_daily_hr['timestamp'] = pd.to_datetime(df_daily_hr['timestamp'])

df_daily_hr['hour'] = df_daily_hr['timestamp'].dt.hour
df_daily_hr['minute'] = df_daily_hr['timestamp'].dt.minute
df_daily_hr['second'] = df_daily_hr['timestamp'].dt.second

df_daily_hr['time_of_day'] = pd.cut(df_daily_hr['hour'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)

# BPM Features
df_daily_hr['rolling_avg_bpm'] = df_daily_hr['bpm'].rolling(window=5).mean() # window can be adjusted
df_daily_hr['bpm_change'] = df_daily_hr['bpm'].diff()

# Quality and Source Features
df_daily_hr = pd.get_dummies(df_daily_hr, columns=['quality', 'source'], drop_first=True)


# Date and Time Features
df_daily_hr['day_of_week'] = df_daily_hr['timestamp'].dt.day_name()
df_daily_hr['is_weekend'] = df_daily_hr['day_of_week'].isin(['Saturday', 'Sunday'])

# Aggregated Features
df_daily_hr['date'] = df_daily_hr['timestamp'].dt.date
daily_aggregates = df_daily_hr.groupby('date').agg({
    'bpm': ['mean', 'min', 'max', 'std']
})

In [6]:
df_daily_hr.head(5)

,timestamp,bpm,restorative,hour,minute,second,time_of_day,rolling_avg_bpm,bpm_change,quality_bad,quality_good,day_of_week,is_weekend,date
0,2021-11-28 16:41:28+00:00,65,False,16,41,28,Afternoon,NaN,NaN,True,False,Sunday,True,2021-11-28
1,2021-11-28 16:41:29+00:00,70,False,16,41,29,Afternoon,NaN,5.0,True,False,Sunday,True,2021-11-28
2,2021-11-28 16:41:30+00:00,75,False,16,41,30,Afternoon,NaN,5.0,True,False,Sunday,True,2021-11-28
3,2021-11-28 16:51:32+00:00,77,False,16,51,32,Afternoon,NaN,2.0,True,False,Sunday,True,2021-11-28
4,2021-11-28 16:51:35+00:00,73,False,16,51,35,Afternoon,72.0,-4.0,True,False,Sunday,True,2021-11-28
